In [ ]:
# !pip install pyts
# !pip install tensorflow
# !pip install numpy
# !pip install pandas
# !pip install optuna
# !pip install optuna-integration[tfkeras]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import optuna
import tensorflow as tf

from pyts.image import GramianAngularField, MarkovTransitionField
from sklearn.preprocessing import MinMaxScaler
from optuna.integration import TFKerasPruningCallback
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
#################
###   Data    ###
#################

bitcoin = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BTC_noscale.csv')
bitcoin.drop(columns=['Crypto', 'Open', 'High', 'Low'], inplace=True)
bitcoin.head()

,Date,Close
0,2018-01-01,13535.0
1,2018-01-02,14770.0
2,2018-01-03,15057.0
3,2018-01-04,14921.0
4,2018-01-05,16828.0


In [ ]:
#######################
#### Input GAF/MTF ####
#######################

btc = bitcoin.iloc[:]

# ------------
# Parameters
# ------------
window_size = 224     # timesteps per sample
image_size = 224      # size for GAF/MTF images
train_ratio = 0.75    # training ratio


# ----------------------
# Step 1 — Normalization
# ----------------------
scaler = MinMaxScaler(feature_range=(0, 1))
btc['close_scaled'] = scaler.fit_transform(btc[['Close']])

# ---------------------------------
# Step 2 — Generate sliding windows
# ---------------------------------
def create_sequences(series, window_size):
    X_seq = []
    y = []
    for i in range(len(series) - window_size):
        X_seq.append(series[i:i+window_size])
        y.append(series[i+window_size])  # predict next value
    return np.array(X_seq), np.array(y)

X_seq, y = create_sequences(btc['close_scaled'].values, window_size)

# ---------------------------------
# Step 3 — GAF & MTF for CNN branch
# ---------------------------------
gasf = GramianAngularField(image_size=image_size, method='summation')
gadf = GramianAngularField(image_size=image_size, method='difference')
mtf = MarkovTransitionField(image_size=image_size)

# Apply transforms to each sequence
X_gasf = gasf.fit_transform(X_seq)
X_gadf = gadf.fit_transform(X_seq)
X_mtf = mtf.fit_transform(X_seq)

# Stack into channels: (samples, H, W, 3)
X_cnn = np.stack((X_gasf, X_gadf, X_mtf), axis=-1)

In [ ]:
n_samples = len(X_seq)
train_end = int(n_samples * train_ratio)

X_cnn_train, X_cnn_test = X_cnn[:train_end], X_cnn[train_end:]
y_train, y_test = y[:train_end], y[train_end:]

# -------------------
# Shapes check
# -------------------
print("CNN train shape:", X_cnn_train.shape)   # (samples, H, W, 3)
print("Targets train shape:", y_train.shape)

CNN train shape: (1314, 224, 224, 3)
Targets train shape: (1314,)


In [ ]:
####################
def objective(trial):
    conv1_filters = trial.suggest_categorical('conv1_filters', [32, 64, 128])
    conv2_filters = trial.suggest_categorical('conv2_filters', [16, 32, 64])
    dense_units = trial.suggest_categorical('dense_units', [64, 128, 256])
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    optimizer_choice = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 30, 200, step=10)


    # Adam/RMSprop
    if optimizer_choice == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    # VGG setup with input from GAF/MTF
    input_layer = Input(shape=(224, 224, 3))
    vgg_base = VGG16(include_top=False, weights='imagenet', input_tensor=input_layer)

    # CNN + Convolution layer
    x = vgg_base.output
    x = Conv2D(conv1_filters, (2, 2), activation=activation)(x)
    x = Conv2D(conv2_filters, (2, 2), activation=activation)(x)
    x = Flatten()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(dense_units, activation=activation)(x)
    output = Dense(1, activation='linear')(x)

    # Build model
    model = Model(inputs=input_layer, outputs=output)

    model.compile(loss='mean_squared_error',
                  optimizer=optimizer,
                  metrics=['mae', RootMeanSquaredError(), 'mape'])

    # Pruning
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    pruning_cb = TFKerasPruningCallback(trial, 'val_loss')

    # Compile the model
    history = model.fit(
        X_cnn_train, y_train,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stop, pruning_cb]
    )

    val_loss = min(history.history['val_loss'])
    return val_loss


In [ ]:
#### Run optuna ####
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2025-09-17 12:25:20,629] A new study created in memory with name: no-name-b8aa4aff-b1fb-4a7c-a7d3-c9383db1736b


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


[I 2025-09-17 12:26:09,841] Trial 0 finished with value: 0.2845718562602997 and parameters: {'conv1_filters': 128, 'conv2_filters': 64, 'dense_units': 64, 'dropout_rate': 0.40819734342713926, 'activation': 'relu', 'optimizer': 'rmsprop', 'learning_rate': 5.201191486830627e-05, 'batch_size': 16, 'epochs': 110}. Best is trial 0 with value: 0.2845718562602997.
[I 2025-09-17 12:27:14,376] Trial 1 finished with value: 0.029393712058663368 and parameters: {'conv1_filters': 32, 'conv2_filters': 64, 'dense_units': 128, 'dropout_rate': 0.24762526623377706, 'activation': 'relu', 'optimizer': 'rmsprop', 'learning_rate': 0.0007294662520741923, 'batch_size': 128, 'epochs': 110}. Best is trial 1 with value: 0.029393712058663368.
[I 2025-09-17 12:27:45,506] Trial 2 finished with value: 0.19582372903823853 and parameters: {'conv1_filters': 32, 'conv2_filters': 64, 'dense_units': 128, 'dropout_rate': 0.27797872376841126, 'activation': 'relu', 'optimizer': 'adam', 'learning_rate': 0.00013529158579293695

In [ ]:
print("Best trial:")
trial = study.best_trial

print(f"  Loss: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Loss: 0.029393712058663368
  Params: 
    conv1_filters: 32
    conv2_filters: 64
    dense_units: 128
    dropout_rate: 0.24762526623377706
    activation: relu
    optimizer: rmsprop
    learning_rate: 0.0007294662520741923
    batch_size: 128
    epochs: 110


In [ ]:
from google.colab import files

results_cnn = study.trials_dataframe()
results_cnn.to_csv('optuna_cnn_results.csv', index=False)

files.download('optuna_cnn_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Best results after 4 runs of the optimisation

# Best trial:
#   Loss: 0.029393712058663368
#   Params:
#     conv1_filters: 32
#     conv2_filters: 64
#     dense_units: 128
#     dropout_rate: 0.24762526623377706
#     activation: relu
#     optimizer: rmsprop
#     learning_rate: 0.0007294662520741923
#     batch_size: 128
#     epochs: 110

# Best trial:
#   Loss: 0.02948707714676857
#   Params:
#     conv1_filters: 128
#     conv2_filters: 64
#     dense_units: 128
#     dropout_rate: 0.29145153405423596
#     activation: tanh
#     optimizer: adam
#     learning_rate: 0.0004047368844376192
#     batch_size: 64
#     epochs: 80

# Best trial:
#   Loss: 0.03042406029999256
#   Params:
#     conv1_filters: 128
#     conv2_filters: 32
#     dense_units: 128
#     dropout_rate: 0.12376764732686152
#     activation: tanh
#     optimizer: rmsprop
#     learning_rate: 0.0006722258813630644
#     batch_size: 64
#     epochs: 120

# Best trial:
#   Loss: 0.09799350053071976
#   Params:
#     conv1_filters: 32
#     conv2_filters: 16
#     dense_units: 128
#     dropout_rate: 0.3929329850769902
#     activation: tanh
#     optimizer: rmsprop
#     learning_rate: 0.0001689385972851931
#     batch_size: 16
#     epochs: 40
